# Dudas y preguntas frecuentes

### Formato de presentación

La presentación de los ejercicios se realizará en una tarea del aula virtual mediante un archivo comprimido. Debe incluir el código completo .py de todos los ejercicios y ficheros auxiliares (siempre que no sean muy pesados), y una memoria en formato pdf o jupyter que incluya una explicación de las solución elegidas, las funciones o trozos de código más importantes, y pantallazos o videos de demostración con imágenes originales. También es conveniente indicar los casos de fallo y el tiempo de cómputo. Lo importante, además de la evaluación, es que os quede un buen documento de referencia para el futuro.

### Si las ventanas no muestran las imágenes

Hace falta `cv.waitKey(ms)` para [refrescar el interfaz](https://docs.opencv.org/4.1.0/d7/dfc/group__highgui.html#ga5628525ad33f52eab17feebcfba38bd7) de opencv. Esto se hace automáticamente dentro del bucle de captura de autoStream(). En cualquier otro caso es necesario llamar a esta función para que se refresquen la ventanas aunque no necesitemos la tecla pulsada.

### Ejercicio FOV

Para la calibración precisa se puede utilizar el código disponible en `code/calibrate`, siguiendo la instrucciones del README. Uno de los resultados que produce es la matriz de cámara $K$, que contiene en la diagonal el valor de $f$:

$$K =\begin{bmatrix}f & s & o_x \\  0 & fr & o_y \\ 0 & 0 & 1\end{bmatrix}$$ 

Debe tener un valor parecido ($\pm 30\%$) al obtenido de forma aproximada.

La matriz de calibración se usará más adelante en los ejercicios de geometría visual.

La *focal length* suele aparecer en los metadatos en las fotos del móvil (pero no en pantallazos de la webcam). Puede consultarse con la aplicación "image viewer" de gnome (eog), [como se ve aquí](../images/demos/focalinfo2.png). El valor de focal length que nos interesa es el equivalente a 35mm (correspondiente a un sensor 36mmx24mm). Nos dice que la foto de este ejemplo tiene una focal equivalente a 52.0mm para un ancho de sensor de 36mm. Resolviendo el triángulo encontramos un FOV de $38.2^{\circ}$. Alternativamente, podemos consultar [una tabla como ésta](https://www.nikonians.org/reviews/fov-tables) en la que el valor más cercano que aparece es 50mm, que corresponde a $39.6^\circ$.

Si en los metadatos aparece la focal length pero no la equivalente a 35mm, entonces necesitamos conocer el tamaño real del sensor, que a veces puede encontrarse en las especificaciones técnicas del teléfono móvil, pero no merece la pena perder mucho tiempo con esto.

### Ejercicio HISTCOL

Se trata de crear una aplicación parecida a la que se muestra en el siguiente pantallazo: La ventana superior es la imagen en vivo de la webcam (o la fuente de video deseada). Cuando se marca un ROI con el ratón se muestran los histogramas (normalizados) de los 3 canales por separado. Y si se pulsa una cierta tecla se guarda el recuadro como un modelo más y se muestra en la ventana "models" de abajo a la izquierda. En este caso vemos que se han guardado ya tres modelos. En todo momento (siempre que haya algún modelo guardado) se comparan los histogramas del ROI actual con los de todos los modelos. Las distancias se muestran arriba a la izquierda. La menor, 0.32, nos indica que el segundo modelo es el más parecido, y se muestra en la ventana "detected". Si la menor distancia es muy grande se puede rechazar la decisión y y mostrar un recuadro negro. La comparación entre histogramas puede hacerse de muchas formas. Una muy sencilla es la suma de diferencias absolutas en cada canal y quedarnos el máximo de los tres canales.

![HISTCOL](../images/demos/HISTCOL.png)

### Ejercicio SIFT

Algunas recomendaciones:

Los objetos deben tener "detalles" para que aparezcan dentro del él suficientes puntos de interés. Hay objetos muy uniformes que producen muy pocos puntos, o salen casi todos por los bordes o el exterior, y entonces no son muy apropiados para este método. Por ejemplo una libreta con tapas negras, o un teléfono móvil, una botella de cristal sin etiqueta, etc. no funcionan bien.

Pulsando una tecla se pueden ir guardando modelos sobre la marcha, pero es conveniente leerlos de una carpeta para agilizar el uso del programa.

Hay que calcular una sola vez los keypoints y descriptores de los modelos. En el bucle de captura solo deben calcularse los de la imagen actual.

Para reducir el tiempo de cálculo se puede trabajar con imágenes me menor resolución (eg. 400x300) y limitar el número de keypoints en las comparaciones (parámetro nfeatures).

Usando autoStream() lo normal es que se produzca un retardo en las imágenes, por lo que es preferible usar la captura con hilo mediante la utilidad Camera de umucv. 

Sí a pesar de todo el proceso va muy lento, se puede efectuar la extracción de keypoints y la comparación con los modelos solo cuando detectemos que la imagen está bastante quieta o cuando pulsemos una tecla.

Si los modelos tienen diferente número de keypoints la comparación debe hacerse teniendo en cuenta el porcentaje de coincidencias, no el valor absoluto.

Se puede rechazar la decisión cuando el porcentaje ganador sea pequeño, cuando el segundo mejor sea parecido al primero, o cuando haya pocas coincidencias en la imagen, entre otras situaciones que dependen de la aplicación.

Cuando no se detecta ningún keypoint los descriptores no se devuelven como un array de dimension 0x128, sino como un valor `None`. Hay que tener cuidado con  esto para que no se produzcan errores en tiempo de ejecución. Esto puede ocurrir cuando la cámara apunta hacia la mesa, o está muy desenfocada.

Os dejo aquí un [vídeo de ejemplo](https://robot.inf.um.es/material/va/sift-demo.mp4) de lo que se puede conseguir sin muchas complicaciones. Se muestra en pequeño el modelo ganador, su porcentaje, y la diferencia con el segundo mejor. Observa que los objetos se reconocen aunque no se vean enteros en la imagen, con diferentes tamaños, con cualquier rotación en el plano de imagen, y con cierta inclinación de perspectiva. Hay también cierta resistencia al desenfoque y a reflejos. (Aunque no se ven en esta breve secuencia, pueden producirse clasificaciones erróneas cuando la escena tiene muchos puntos y no hay ningún modelo conocido.) 